
# Few-Shot Prompting — MVP (Sentiment Classification)

**Goal:** Provide a few labeled examples to guide the model.  
Assumes `OPENAI_API_KEY` is set.


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
import json

In [5]:
# Manually write out the prompt string with the examples and use PromptTemplate

manual_prompt_str = """You are a concise sentiment analyst.
Decide whether the sentiment is "Positive" or "Negative".

Here are examples:
Input: I love the new update!
Output: {{"label":"Positive","rationale":"Expresses approval"}}
---
Input: This is a terrible experience.
Output: {{"label":"Negative","rationale":"States terrible"}}
---
Input: Not bad for a budget phone.
Output: {{"label":"Positive","rationale":"Negation implies acceptable"}}
---
Now classify the sentiment for the following input.
Input: {user_text}
Output:"""

manual_prompt = PromptTemplate.from_template(manual_prompt_str)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

chain = manual_prompt | llm

chain.invoke({"user_text": "Dancing in the rain is fun!"})



AIMessage(content='{"label":"Positive","rationale":"Expresses enjoyment and fun"}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 121, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CPXaL4SZGEoHPidGe7KV5LtKlrb5U', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9ee29980-a0c6-4d6a-ab50-6827ca389057-0', usage_metadata={'input_tokens': 121, 'output_tokens': 14, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

example_prompt = PromptTemplate.from_template(
    """Input: {text}
Output: {output_json}""")

examples = [
    {"text": "I love the new update!", "output_json": '{{"label":"Positive","rationale":"Expresses approval"}}'},
    {"text": "This is a terrible experience.", "output_json": '{{"label":"Negative","rationale":"States terrible"}}'},
    {"text": "Not bad for a budget phone.", "output_json": '{{"label":"Positive","rationale":"Negation implies acceptable"}}'}
]

prefix = """You are a concise sentiment analyst.
Decide whether the sentiment is "Positive" or "Negative".

Here are examples:
"""

suffix = """Now classify the sentiment for the following input.
Input: {user_text}
Output:"""

fs_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["user_text"],
    example_separator="\n---\n",
)

In [7]:
print(fs_template.invoke({"user_text": "Dancing in the rain is fun!"}).text)

You are a concise sentiment analyst.
Decide whether the sentiment is "Positive" or "Negative".

Here are examples:

---
Input: I love the new update!
Output: {"label":"Positive","rationale":"Expresses approval"}
---
Input: This is a terrible experience.
Output: {"label":"Negative","rationale":"States terrible"}
---
Input: Not bad for a budget phone.
Output: {"label":"Positive","rationale":"Negation implies acceptable"}
---
Now classify the sentiment for the following input.
Input: Dancing in the rain is fun!
Output:


In [8]:
chain = fs_template | llm

In [9]:

def few_shot_classify(text: str):
    raw = chain.invoke({"user_text": text}).content
    try:
        return json.loads(raw)
    except Exception:
        return {"label": "Unknown", "rationale": raw}


### Compare on trickier inputs

In [10]:

tricky = [
    "I hate to admit it, but this is actually pretty good.",
    "The camera is fine, but the battery life is awful.",
    "I can't say it's bad."
]
for s in tricky:
    print("\nINPUT:", s)
    print(json.dumps(few_shot_classify(s), indent=2))



INPUT: I hate to admit it, but this is actually pretty good.
{
  "label": "Positive",
  "rationale": "Contradiction indicates unexpected approval"
}

INPUT: The camera is fine, but the battery life is awful.
{
  "label": "Negative",
  "rationale": "Mentions awful battery life despite a neutral comment about the camera"
}

INPUT: I can't say it's bad.
{
  "label": "Positive",
  "rationale": "Negation implies a positive or neutral sentiment"
}


### Try your own

In [ ]:

my_text = "Type here"
print(json.dumps(few_shot_classify(my_text), indent=2))
